In [12]:
import os
from dotenv import load_dotenv
load_dotenv()  #load all the environment variables

True

In [13]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [14]:
import json
from langchain.schema import Document

# Load JSON file
with open("pharmakon_products.json", "r", encoding="utf-8") as f:
    products = json.load(f)

#  Convert each product description into a single Document
# the embedding model will be for description, not for the whole product details , but we have metadata for each product
docs = []
for product in products:
    docs.append(Document(
        page_content=product["product_description"],  # full description in one chunk
        metadata={
            "name": product["product_name"],
            "link": product["product_link"],
            "price": product["product_price"]
        }
    ))

#  Check first document
print(docs[3])


page_content='Composition
Menthol, arginine, Allantoin, caffeine, Carnitine, Glycerin, Carbomer, Triethanolamine, CMC , PVP , Monopropylene glycol , Polysorbate 20 , aloe vera, Chlorhexidine , eugenol, methyl paraben, propyl paraben, sugar, Parfum, Aqua.
properties
Skout Gel works with absorption effect technology. It is quick to be absorbed from the penis skin , so the desensitizing effect will not transfer to your partner. Skout Gelenables you to keep pleasurable sexual action , while delaying your semen from coming out. Skout Gel allows you to last more and more time enjoying sex. Skout Gel creates a warming sensation as vessels dilate and capillaries expand. Skout Gel provides the vigor and vitality of men. Skout Gel provides unmatched sexual desire. Skout Gel increases sexual ability for men. Skout Gel provides nourishment to improve the person’s energy level and mood , thus giving an extra pleasure. Skout Gel improves the circulation in the penile tissue as it increases blood flo

In [15]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

#  Initialize the embedding model
#  Initialize the embedding model
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

#  Create a Chroma vector store from the docs
if not os.path.exists("./chroma_db"):
    vectordb = Chroma.from_documents(
        documents=docs,       # your list of Document objects
        embedding=embedding_model,
        persist_directory="./chroma_db"  # folder to save vector database
    )

    #  Persist the database to disk
    vectordb.persist()
else:
    if os.path.exists("./chroma_db"):
    # Load existing database
        vectordb = Chroma(
            persist_directory="./chroma_db",
            embedding_function=embedding_model
        )
        print("Loaded existing vector database.")


print("Vector database created and persisted successfully!")


Loaded existing vector database.
Vector database created and persisted successfully!


In [16]:
# def print_results(results):
#     for i, doc in enumerate(results, 1):
#         print(f"Result {i}:")
#         print("Product Name:", doc.metadata["name"])
#         print("Link:", doc.metadata["link"])
#         print("Price:", doc.metadata["price"])
#         print("Description:", doc.page_content[:300])  # preview first 300 chars
#         print("----------------------------")
def print_results(results):
    for i, (doc, score) in enumerate(results, 1):  # unpack tuple
        print(f"Result {i} (Confidence: {score:.2f}):")
        print("Product Name:", doc.metadata["name"])
        print("Link:", doc.metadata["link"])
        print("Price:", doc.metadata["price"])
        print("Description:", doc.page_content[:300])
        print("----------------------------")

        

In [17]:
def query_vector_db(query, k=1):
    """
    Query the vector database and return top k results.
    
    Args:
        query (str): The search query.
        k (int): Number of top results to return.
    
    Returns:
        list: List of Document objects with metadata.
    """
    results_with_score = vectordb.similarity_search_with_score(query, k=k)
    threshold = 0.9
    filtered_results = [(doc, score) for doc, score in results_with_score if score >= threshold]
    if not filtered_results:
        print("No results found above the threshold.")
        filtered_results = []  # Return empty list if no results meet the threshold
        return 

    return filtered_results

In [18]:
# #  Load the persisted Chroma database
# embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
# vectordb = Chroma(
#     persist_directory="./chroma_db",
#     embedding_function=embedding_model
# )

In [47]:
query =  "sunscren for oily skin"  # Define your query
results = query_vector_db(query,k=2) 

#
print_results(results)

Result 1 (Confidence: 1.09):
Product Name: Gawhara Sunscreen Cream SPF +30
Link: https://pharmakonegypt.org/product/gawhara-sunscreen-cream-spf-30-2/
Price: EGP180
Description: Gawhara sunscreen cream SPF +30
Sunscreen cream
----------------------------
Result 2 (Confidence: 1.09):
Product Name: Gawhara Sunscreen Cream SPF +30
Link: https://pharmakonegypt.org/product/gawhara-sunscreen-cream-spf-30-2/
Price: EGP180
Description: Gawhara sunscreen cream SPF +30
Sunscreen cream
----------------------------


In [48]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("""
You are a helpful medical advisor assistant.
A customer has described their condition or symptoms.
You must choose the most relevant products from the retrieved list below and if there is no relevants say no products available, display the product price and its link.

Customer Query: {input}

Retrieved Products:
{context}

Instructions:
- Recommend the most relevant product.
- Explain briefly why the product matches the query.
- If no suitable product is found, clearly say "No relevant product found."
- Keep the answer short and professional.
- Provide the product name, price, and link from provided context metadata.
""")


In [50]:
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
def augmented_generation():
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    document_chain = create_stuff_documents_chain(llm, prompt_template)

    # Wrap with retrieval
    context = query_vector_db(query, k=3)

    final_prompt = prompt_template.format(context=context, input=query)

    response = llm.invoke(final_prompt)
    print(response.content)
    return response
response = augmented_generation()

The most relevant product for oily skin is:

**Product Name:** Gawhara Sunscreen Cream SPF +30  
**Price:** EGP 180  
**Link:** [Gawhara Sunscreen Cream SPF +30](https://pharmakonegypt.org/product/gawhara-sunscreen-cream-spf-30-2/)

This sunscreen is suitable for oily skin as it provides sun protection while being lightweight, which helps prevent excess oiliness.


**Recommended Product:** Gawhara sunscreen cream SPF +30  
**Price:** Not specified  
**Link:** Not provided  

This product is suitable for oily skin as it is a sunscreen cream with SPF +30, which helps protect the skin from UV rays while being lightweight, making it ideal for oily skin types. 

If you need more specific details or a different product, please let me know!
